## Requirements
What are all the requirements for the client? 
0. Connect the the datasite
1. Be able to interact with different services of RDS, e.g. `code`, `dataset`, `jobs`?
2. Connect to multiple datasites?
3. In the future we will have authentication and other services, we need to make it easy to add new services to the client / extend client's functionalities

```python
import syft_rds as sy
import syft_rds.RDSClient as RDSClient

client: RDSClient = sy.connect(datasite="rasswanth@openmined.org")

client.services  # list all services like dataset, jobs, functions...

client.dataset.create(  # should call the create() method of the dataset service
    name="My Very Private Dataset",
    summary="My dataset is very private.",  # optional
    description_path="~/.../description.md",  # optional
    path="...",
    mock_path="...",
    # data_loader=None,
)

dataset: Dataset = client.dataset.get_by_name(name="My Very Private Dataset")
```

## PseudoCode


```python
class Dataset:
    def create(name: str, description: str, path: str, mock_path: str):
        """
        - Client side: It creates a Dataset object with data as file paths linked to by `path` and `mock_path` fields
        - RPC: creates an RPC request "syft://khoa@openmined.org/api_data/dataset/rpc/create" with the Dataset object as the body, the request first stays locally in the client SyftBox folder
        - RPC request gets synced by the cache server when the client runs SyftBox in the background
        - On the server, the Dataset object is created and saved to the database
        """
        # client side
        dataset = Dataset(
            name="my_dataset",
            description="This is a dataset",
            path="SyftBoxStage/datasites/khoa@openmined.org/mock.csv",  # files need to be at right place and we check them exist?? 
            mock_path="SyftBoxStage/datasites/khoa@openmined.org/real.csv"
        )
        rpc_url: SyftURL = create_url("dataset", "rpc", "create")  # something like syft://khoa@openmined.org/api_data/dataset/rpc/create
        future: SyftFuture = rpc.send(
            client=client,
            url=rpc_url,
            headers={},
            body=dataset.model_dump_json(), # "{"name": "my_dataset"}"
            expiry="15m",
            cache=True,
        )
        # creates a <id>.request file in the "khoa@openmined.org/api_data/dataset/rpc/create"

        # server: @box.on_request("/dataset/create") will see the request, reconstructs the 
        # Dataset object from the request body, saves the Dataset object to a db / file
        
        # do we save the Dataset obj locally (on the client side)?

        return future  # returned to the client

    def get_all() -> list[Dataset]:
        """
        Creates an RPC request to get all datasites from the server
        The request body should not contains anything since it will be a GET request


        If dataset object metadata are saved locally (e.g. created by Rasswanth) and get synced down to my SyftBox folder (Khoa's),
        # then Khoa don't need to create an RPC request to get all datasets again, as Khoa can just get them from local
        """
        rpc_url: SyftURL = create_url("dataset", "rpc", "get_all")  # something like syft://<client.remote_datasite>/api_data/dataset/rpc/get_all
        # future: SyftFuture = rpc.send(
        #     client=client,
        #     url=rpc_url,
        #     headers={},
        #     body={},
        #     expiry="15m",
        #     cache=True,
        # )
        if Path("client.remote_datasite>/api_data/datasets") is dir():
            return len(Path("client.remote_datasite>/api_data/datasets"))


        # server side: @box.on_request("/dataset/get_all") will see the request, retrieves all the datasets from the db
        # and returns that response to the client

        return future
        

        # server
        box = SyftEvents("test_app")
        @box.on_request("/dataset/get_all")
        def get_all_datasets(request):
            datasets = DB.dataset.get_all(client=request.client)
```



```
dataset
├── rpc
│   ├── create
│   ├── get
│   ├── list
│   ├── update
│   └── delete
code

```

## Client and Server Facing

**Client-facing RDSCLient**: Interacts with local `SyftBox` and Jupyter Notebook user?
**Server-facing RDSCLient**: Interacts with the cache server through RPC requests?

we can implement like this
```python
class Job:
	def approve(job_id):
		# client-facing logic
    	prompt = input("Are you sure you really want to approve")
    	if prompt == "no":
		    return

        # server-facing logic
        rpc.make_url()
        future = rpc.send()
        if blocking:
            res = future.wait()
            return res
        else:
            return future
```

but below is better (wrap all server-facing logic to API class)
```python
class API:
    class Job:
        def approve(job_id):
            # Pure CRUD/RPC calls, no business logic
            rpc.make_url()
            future = rpc.send()
            if blocking:
                res = future.wait()
                return res
            else:
                return future

class Job:
	def approve(job_id):
    	# client-facing logic
    	prompt = input("Are you sure you really want to approve this job")
    	if prompt == "no":
    		return
        return self.api.job.approve(job_id) # server-facing logic implemented by API
```

So in general, we want usage:
```python
client: RDSClient = sy.connect(datasite="rasswanth@openmined.org")
client.dataset.create(...) # this calls client.api.datasset.create()
client.job.approve(...) # this calls client.api.job.approve()
# generic way
client.service.service_function(...) # this calls client.api.service.service_function()
```
So the classes may look like (this is the Facade pattern):
```python
def connect(host: str):
    return RDSClient(host)

class RDSClient:
    def __init__(self, host: str):
        self.host = host
        self._api = API()
        self.service = Service(self._api)

class API:
    def __init__(self):
        self.service = ServiceAPI(self)

class Service:
    def __init__(self, api):
        self.api = API()
    
    def service_function(self, args):
        # client facing logic
        ...
        # server-facing logic
        self._api.service.service_function(args)

class ServiceAPI:
    def __init__(self, api):
        self.api = API()
    
    def service_function(self, args):
        # only server-facing logic
        rpc.makeURL(...)
        rpc.send(...)
```

### RDSClient stub implementation using the Facade Pattern

```python
def connect(host: str):
    return RDSClient(host)

class RDSClient():
    def __init__(self, host: str):
        self.host = host
        self._api = API(self.host)
        self.job = Job(self._api)
        self.dataset = Dataset(self._api)

class API:
    def __init__(self, host: str):
        """Internal Server-facing API implementation."""
        print(f"Init APIs for remote datasite {host}")
        self.host = host
        self.job = JobAPI(self)

class Job:
    """Client-facing logic for Job"""
    def __init__(self, api: API):
        self._api = api

    def approve(self, job_id):
        # client-facing part
        prompt = input(f"Are you sure you want to approve job {job_id}? (yes/no): ")
        if prompt.lower() != "yes":
            print("Job approval cancelled")
            return None

        # server-facing wrapped by the _api
        return self._api.job.approve(job_id)

class JobAPI:
    """
    Implements Server-facing logic for Job
    """
    def __init__(self, api: API):
        self.api = api

    def approve(self, job_id):
        print(f"Sending RPC request to server for job {job_id}")
        return True

class Dataset:
    pass

class DatasetAPI:
    pass


if __name__ == "__main__":
    client: RDSClient = connect("rasswanth@openmined.org")
    result = client.job.approve("job123")
    print(result)
```

## Questions

Q1: How RDSClient relates to SyftBoxClient?

## Dataset Exploration

TODO: 
Create RDSClient docs:
- Requirements
- PseudoCode
    - Some functions need RPC
    - Some functions don't need RPC. These should be asked
- Server-facing and Client-facing parts
- Questions